## Importing Dataset

In [2]:
import os

In [3]:
def extract():
    text = ''
    path = '/kaggle/input/marvel-cinematic-universe-dialogue-dataset'
    for file in os.listdir(path):
        path_file = os.path.join(path + '/', file)
        with open(path_file, 'r', errors= 'ignore') as f:
            text += f.read()
    return text

In [4]:
text = extract()

In [5]:
text[:100]

'(BUCKY SCREAMING)\n(CONTINUES SCREAMING)\n- (KARPOV SPEAKING RUSSIAN) - (PANTING)\nLonging\nRusted\nSeven'

# Text Processing

In [6]:
class Preprocess():
    def __init__(self, text):
        super().__init__()
        self.text = text
        
    def create_vocab(self):
        vocab = sorted(list(set(self.text)))
        self.stoi = {s: i for i, s in enumerate(vocab)}
        self.itos = {i: s for s, i in self.stoi.items()}
        return vocab, len(vocab), self.stoi, self.itos
    
    def encode(self, string):
        return [self.stoi[char] for char in string]
    
    def decode(self, array):
        return ''.join(self.itos[idx] for idx in array)

In [7]:
text_processor = Preprocess(text)
vocab, vocab_size, stoi, itos = text_processor.create_vocab()

In [8]:
print(text_processor.encode('hello'))
text_processor.decode(text_processor.encode('hello'))

[61, 58, 65, 65, 68]


'hello'

# Set Device

In [9]:
import torch

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Split Dataset

In [11]:
import torch.nn as nn

In [12]:
data = torch.tensor(text_processor.encode(text), dtype = torch.long)
data[:50], len(data)

(tensor([ 8, 29, 48, 30, 38, 52,  1, 46, 30, 45, 32, 28, 40, 36, 41, 34,  9,  0,
          8, 30, 42, 41, 47, 36, 41, 48, 32, 46,  1, 46, 30, 45, 32, 28, 40, 36,
         41, 34,  9,  0, 12,  1,  8, 38, 28, 45, 43, 42, 49,  1]),
 1147310)

In [13]:
n = int(0.8 * len(data))
train = data[:n]
val = data[n:]
len(train), len(val)

(917848, 229462)

In [14]:
def split(type):
    data = train if type == 'train' else val
    idx = torch.randint(len(data) - block_size, (batch_size, ))
    X = torch.stack([data[i: i + block_size] for i in idx])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in idx])
    X, y = X.to(device), y.to(device)
    return X, y

In [15]:
batch_size = 128
block_size = 256
max_iters = 1
eval_interval = 500
learning_rate = 1e-3
eval_iters = 250
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
window_size = 128


In [16]:
Xtr, ytr = split('train')
Xtr.shape, ytr.shape

(torch.Size([128, 256]), torch.Size([128, 256]))

# Define Error List

In [17]:
import torch.nn.functional as F

In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for splits in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = split(splits)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[splits] = losses.mean()
    model.train()
    return out

# Create Model

In [19]:
class LongformerAttention(nn.Module):
    """ Longformer-style self-attention with local and global attention """

    def __init__(self, head_size, window_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.window_size = window_size
#         self.global_attention_indices = global_attention_indices

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)
        v = self.value(x) # (B, T, C)

        # Local attention
        attention_scores = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, T)
        attention_scores = self._mask_local_attention(attention_scores)
        attention_probs = F.softmax(attention_scores, dim=-1)
        attention_probs = self.dropout(attention_probs)
        local_out = attention_probs @ v  # (B, T, C)

        # Global attention (only for specific indices)
#         global_out = self._apply_global_attention(q, k, v, x)

        # Combine local and global attention
        out = local_out
        return out

    def _mask_local_attention(self, attention_scores):
        mask = torch.full_like(attention_scores, float('-inf'))
        for i in range(self.window_size, attention_scores.size(-1)):
            mask[:, i-self.window_size:i+self.window_size+1, i] = 0
        return attention_scores.masked_fill(mask == float('-inf'), float('-inf'))

#     def _apply_global_attention(self, q, k, v, x):
#         global_out = torch.zeros_like(x)
#         for idx in self.global_attention_indices:
#             global_out[:, idx, :] = (q[:, idx, :] @ k.transpose(-2, -1) @ v).squeeze(1)
#         return global_out


In [20]:
class MultiHeadLongformerAttention(nn.Module):
    def __init__(self, num_heads, head_size, window_size):
        super().__init__()
        self.heads = nn.ModuleList(
            [LongformerAttention(head_size, window_size) for _ in range(num_heads)]
        )
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [21]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [22]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head, window_size):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadLongformerAttention(n_head, head_size, window_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [23]:
import torch
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head, window_size=window_size)
              for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
#         print(B,T)
        tok_emb = self.token_embedding_table(idx)
#         print(tok_emb)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
#         print(pos_emb)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        # Debugging Prints
#         print(f"tok_emb: {tok_emb.mean()}, min: {tok_emb.min()}, max: {tok_emb.max()}")
#         print(f"pos_emb: {pos_emb.mean()}, min: {pos_emb.min()}, max: {pos_emb.max()}")
#         print(f"x after blocks: {x.mean()}, min: {x.min()}, max: {x.max()}")
#         print(f"logits: {logits.mean()}, min: {logits.min()}, max: {logits.max()}")

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    


    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            
            print("loss", loss)
            logits = logits[:, -1, :] # becomes (B, C)
            
            probs = F.softmax(logits, dim=-1) # (B, C)
            print(probs)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [24]:
model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.80354 M parameters


# Train the Model

In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = split('train')
    logits, loss = model(xb, yb)
    print(logits)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.5283, val loss 4.5259
tensor([[ 0.4922, -0.1384, -0.3389,  ...,  0.1342,  0.8168,  0.5678],
        [-0.1534,  0.7499,  0.3999,  ..., -0.4580, -0.0597, -0.1686],
        [-0.2127,  0.7754,  1.0429,  ...,  0.1442, -0.2845,  0.4340],
        ...,
        [-0.9298,  0.1860,  0.5714,  ...,  0.0619,  0.9222,  1.3364],
        [ 0.4938,  0.3355,  0.2709,  ...,  0.5943,  1.3439,  0.1290],
        [-0.2462,  0.0217, -0.6837,  ..., -0.2163,  0.3184,  0.8068]],
       device='cuda:0', grad_fn=<ViewBackward0>)


# Generate Text

In [27]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(context)
m.generate(context, max_new_tokens = 100)

tensor([[0]], device='cuda:0')
loss None
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
# def generate(num_words):
def generate(num_words):
#     context = torch.zeros((1, 1), dtype=torch.long, device=device)
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    idx = m.generate(context, max_new_tokens = num_words)
#     print(text_processor.decode()[0].tolist()))
    print(idx)

In [ ]:
generate(1)

In [ ]:
class LongformerAttention(nn.Module):
    """ Longformer-style self-attention with local and global attention """

    def __init__(self, head_size, window_size, global_attention_indices):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.window_size = window_size
        self.global_attention_indices = global_attention_indices

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)
        v = self.value(x) # (B, T, C)

        # Local attention
        attention_scores = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, T)
        attention_scores = self._mask_local_attention(attention_scores)
        attention_probs = F.softmax(attention_scores, dim=-1)
        attention_probs = self.dropout(attention_probs)
        local_out = attention_probs @ v  # (B, T, C)

        # Global attention (only for specific indices)
        global_out = self._apply_global_attention(q, k, v, x)

        # Combine local and global attention
        out = local_out + global_out
        return out

    def _mask_local_attention(self, attention_scores):
        mask = torch.full_like(attention_scores, float('-inf'))
        for i in range(self.window_size, attention_scores.size(-1)):
            mask[:, i-self.window_size:i+self.window_size+1, i] = 0
        return attention_scores.masked_fill(mask == float('-inf'), float('-inf'))

    def _apply_global_attention(self, q, k, v, x):
        global_out = torch.zeros_like(x)
        for idx in self.global_attention_indices:
            global_out[:, idx, :] = (q[:, idx, :] @ k.transpose(-2, -1) @ v).squeeze(1)
        return global_out

class MultiHeadLongformerAttention(nn.Module):
    """ Multiple heads of Longformer-style attention """

    def __init__(self, num_heads, head_size, window_size, global_attention_indices):
        super().__init__()
        self.heads = nn.ModuleList(
            [LongformerAttention(head_size, window_size, global_attention_indices) for _ in range(num_heads)]
        )
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class Block(nn.Module):
    """ Transformer block with Longformer attention """

    def __init__(self, n_embd, n_head, window_size, global_attention_indices):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadLongformerAttention(n_head, head_size, window_size, global_attention_indices)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head, window_size=window_size, global_attention_indices=global_attention_indices)
              for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))  # (T, C)
        x = tok_emb + pos_emb  # (B, T, C)
        x = self.blocks(x)  # (B, T, C)
        x = self.ln_f(x)  # (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)  # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = split('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()